In [2]:
# Importations nécessaires
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.initializers import GlorotUniform
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


# Charger les données
df = pd.read_csv('Miles_Traveled.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
df = df.resample('MS').mean()
df = df.fillna(method='ffill')


In [3]:
from keras.models import model_from_json
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.initializers import GlorotUniform
from keras.preprocessing.sequence import TimeseriesGenerator


In [5]:

# Configuration des données
n_input = 12
n_features = 1

# Séparer les données en ensembles d'entraînement et de test
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Mise à l'échelle des données
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Création du générateur de séries temporelles
train_generator = TimeseriesGenerator(
    train_scaled, train_scaled, length=n_input, batch_size=1)

# Création du modèle LSTM
model = Sequential()
model.add(LSTM(150, activation='tanh', input_shape=(
    n_input, n_features), kernel_initializer=GlorotUniform()))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
history = model.fit(train_generator, epochs=30)

# Prédictions
test_generator = TimeseriesGenerator(
    test_scaled, test_scaled, length=n_input, batch_size=1)
y_pred = model.predict(test_generator)

# Inverser la mise à l'échelle pour obtenir les vraies valeurs
y_pred_original = scaler.inverse_transform(y_pred)
y_test_original = scaler.inverse_transform(test_scaled[n_input:])

# Calcul des métriques
mse = mean_squared_error(y_test_original, y_pred_original)
mape = mean_absolute_percentage_error(y_test_original, y_pred_original)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Percentage Error (MAPE):", mape)


Epoch 1/30
458/458 [==============================] - 5s 5ms/step - loss: 0.0105
Epoch 2/30
458/458 [==============================] - 2s 5ms/step - loss: 0.0058
Epoch 3/30
458/458 [==============================] - 2s 4ms/step - loss: 0.0038
Epoch 4/30
458/458 [==============================] - 2s 5ms/step - loss: 0.0035
Epoch 5/30
458/458 [==============================] - 2s 5ms/step - loss: 0.0022
Epoch 6/30
458/458 [==============================] - 2s 5ms/step - loss: 0.0027
Epoch 7/30
458/458 [==============================] - 2s 5ms/step - loss: 0.0027
Epoch 8/30
458/458 [==============================] - 2s 5ms/step - loss: 0.0023
Epoch 9/30
458/458 [==============================] - 2s 4ms/step - loss: 0.0023
Epoch 10/30
458/458 [==============================] - 2s 4ms/step - loss: 0.0025
Epoch 11/30
458/458 [==============================] - 2s 4ms/step - loss: 0.0026
Epoch 12/30
458/458 [==============================] - 2s 4ms/step - loss: 0.0027
Epoch 13/30
458/458 [====

In [8]:
# Sauvegarder l'architecture du modèle au format JSON
model_json = model.to_json()
with open("model_LSTM.json", "w") as json_file:
    json_file.write(model_json)

# Sauvegarder les poids du modèle au format HDF5
model.save_weights("model_LSTM_weights.h5")

from keras.models import model_from_json

# Charger l'architecture du modèle à partir du fichier JSON
with open("model_LSTM.json", "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)

# Charger les poids du modèle à partir du fichier HDF5
loaded_model.load_weights("model_LSTM_weights.h5")


In [10]:
# Sélectionnez les derniers points de données de l'ensemble d'entraînement
first_eval_batch = train_scaled[-n_input:]

# Redimensionnez le lot pour correspondre au format attendu par le modèle
# Le nouveau format doit être [1, n_input, n_features]
first_eval_batch = first_eval_batch.reshape((1, n_input, n_features))

# Maintenant, vous pouvez utiliser ce lot pour faire une première prédiction
first_pred = model.predict(first_eval_batch)



1/1 [==============================] - 1s 858ms/step


In [11]:
# Sélectionnez les derniers points de données de l'ensemble d'entraînement
last_train_batch = train_scaled[-n_input:]

# Redimensionnez le lot pour correspondre au format attendu par le modèle
# Le format doit être [1, n_input, n_features]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))

# Faire la prédiction pour le premier point de l'ensemble de test
first_test_prediction = model.predict(last_train_batch)

# Appliquer la transformation inverse pour obtenir la valeur originale
first_test_prediction_original = scaler.inverse_transform(first_test_prediction)

print("First test prediction (original scale):", first_test_prediction_original)


1/1 [==============================] - 0s 36ms/step
First test prediction (original scale): [[228815.75]]


In [12]:
# FORECAST USING RNN MODEL

test_predictions = []
first_eval_batch = train_scaled[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    # Obtenir la prédiction actuelle
    current_pred = model.predict(current_batch)[0]

    # Ajouter la prédiction à la liste des prédictions
    test_predictions.append(current_pred)

    # Mettre à jour le lot pour inclure la nouvelle prédiction
    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

    # Imprimez pour comprendre les dimensions et les valeurs
    print(f"Prédiction {i}: {current_pred}")
    print(f"Dimensions actuelles du lot: {current_batch.shape}")

# Appliquer la transformation inverse pour obtenir les valeurs originales
test_predictions_original = scaler.inverse_transform(test_predictions)

# Imprimer les prédictions finales
print("Prédictions finales (échelle originale):")
print(test_predictions_original)


test_predictions_original = scaler.inverse_transform(test_predictions)

# Assurez-vous que la partie mise à l'échelle de test commence à partir de l'indice 'n_input'
y_test_original = scaler.inverse_transform(test_scaled[n_input:])

# Comparez maintenant 'y_test_original' avec 'test_predictions_original'

# Tronquer test_scaled pour correspondre à la longueur des prédictions
# Assurez-vous que la longueur de test_scaled est égale à n_input + longueur de y_test_original_truncated
test_scaled_adjusted = test_scaled[-(n_input + len(y_test_original_truncated)):]

# Générer les prédictions en utilisant test_scaled_adjusted
# Votre code de prédiction ici...

# Assurez-vous maintenant que la longueur des prédictions correspond à celle de y_test_original_truncated
test_predictions_adjusted = test_predictions_original[-len(y_test_original_truncated):]

# Calculer les métriques
mse = mean_squared_error(y_test_original_truncated, test_predictions_adjusted)
mae = mean_absolute_error(y_test_original_truncated, test_predictions_adjusted)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)





1/1 [==============================] - 0s 29ms/step
Prédiction 0: [0.78115076]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 34ms/step
Prédiction 1: [0.8002224]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 33ms/step
Prédiction 2: [0.8235569]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 28ms/step
Prédiction 3: [0.83758074]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 30ms/step
Prédiction 4: [0.8414086]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 39ms/step
Prédiction 5: [0.83165526]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 28ms/step
Prédiction 6: [0.81191933]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 28ms/step
Prédiction 7: [0.78909767]
Dimensions actuelles du lot: (1, 12, 1)
1/1 [==============================] - 0s 29ms/step

In [24]:
import pandas as pd

# Ajuster la longueur de test_predictions_original pour qu'elle corresponde à celle de y_test_original
test_predictions_adjusted = test_predictions_original[-len(y_test_original):]

# Créer un DataFrame avec les valeurs réelles et les prédictions
results_df = pd.DataFrame({
    'Real Values': y_test_original.ravel(),  # Convertir en un tableau 1D si nécessaire
    'Predictions': test_predictions_adjusted.ravel()  # De même, convertir en 1D si nécessaire
})

print(results_df)



     Real Values    Predictions
0       254238.0  213815.530032
1       253936.0  217840.486378
2       256927.0  222141.900756
3       260083.0  224988.696579
4       265315.0  225531.205761
..           ...            ...
101     286608.0  128699.455809
102     260595.0  125484.185956
103     282174.0  121559.717802
104     258590.0  117716.295685
105     268413.0  114797.162801

[106 rows x 2 columns]
